<a href="https://colab.research.google.com/github/CoryTee/JaccardDistanceTweets/blob/master/notebooks/jaccard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import pandas as pd
import numpy as np

def jaccard_dist(setA, setB):
  """
  Computes the Jaccard Distance of two sample sets (A and B) which measures 
  dissimilarity between them. It is defined as the difference of the sizes of 
  the union and the intersection of two sets divided by the size of the union 
  of the sets.
  
  JD(a, b) = (|a_uion_b| - |a_intersect_b|)/ |a_union_b|
  
  How to interpret the result:
    -Small if the tweets are similar
    -Large if the tweets are not similar
    -0 if the tweets have the same words (not counting duplicates or ordering)
    -1 if they are completely different (i.e. no overlapping words)
    
  http://en.wikipedia.org/wiki/Jaccard_index
  """
  
  a_union_b = len(setA.union(setB))
  a_intersect_b = len(setA.intersection(setB))
  
  return (a_union_b - a_intersect_b) / a_union_b
 
  
# Testing values and code
test_string_a = set('the long march'.split())
test_string_b = set('ides of march'.split())
test_jaccard_dist = 0.8 

if jaccard_dist(setA, setB) == test_jaccard_dist:
  print("It works")
else:
  print("You broke something")
  
  
  

It works


In [62]:
import pandas as pd
import numpy as np

data_url = 'file://localhost/content/Tweets.json'


tweets = pd.read_json(data_url, lines=True, orient='records')

#tweets.shape
tweets.columns
#tweets.info()
#tweets.count()


# Split tweet text data using spaces

#setA = set(tweets.loc[[0], ['text']])
#setB = set(tweets.loc[[1], ['text']])
print()

0.8
